In [ ]:
import pandas as pd
import numpy as np
from sklearn.datasets import make_classification, make_blobs
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cross_validation import train_test_split
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use("fivethirtyeight")
from IPython.display import Image

# <b>K-Nearest Neighbors</b>

    - Known as the "easy" machine learning model
    - Classifies an event based on its closest relatives in the data the model has been trained on. Hence the term "Nearest Neighbors". K = number of neighbors.
    - Known as a voting classifier because n neighbors vote for the classification.
    - Uses Euclidean Distance to calculate similarity.
    - Pros: Fast, intuitive, easy to interpret, ability to make probabilities.
    - Cons: Poor at handling many features, especially "noisy" features because it treats every feature equally. Not good with small sample sizes. Usually requires scaling.

In [ ]:
Image("EuclideanDistanceGraphic.jpg")
#Source: Analytics Vidya

### Make fake data 

In [ ]:
data = make_classification(n_samples=200,
                           n_features=2,
                           n_classes=2,
                           n_informative=2,
                           n_redundant=0,
                            class_sep=.75,
                           random_state=5)

In [ ]:
X = data[0]
y = data[1]

In [ ]:
plt.figure(figsize=(15,11))
plt.scatter(X[:,0], X[:,1], c=y, s=250)
plt.xlabel("Feature One")
plt.ylabel("Feature Two")
plt.xlim(-3,3)
plt.ylim(-3,3);

### Train a KNN model using 3 neighbors

In [ ]:
#intialize model and set n_neighbors equal to 3
knn3 = KNeighborsClassifier(n_neighbors=3)
#Fit the model on the "fake data"
knn3.fit(X,y)
#Find the accuracy score of the model on the data
score3 = float(knn3.score(X,y))
print "The model accurately labelled {:.2f} percent of the data".format(score3*100)

### Now with 5 neighbors

In [ ]:
#intialize model and set n_neighbors equal to 5
knn5 = KNeighborsClassifier(n_neighbors=5)
#Fit the model on the "fake data"
knn5.fit(X,y)
#Find the accuracy score of the model on the data
score5 = float(knn5.score(X,y))
print "The model accurately labelled {:.2f} percent of the data".format(score5*100)

### Apply model on a new point

In [ ]:
new_data = np.asarray([0.18,0.15]).reshape(1,-1)
pred3 = knn3.predict(new_data)
pred5 = knn5.predict(new_data)
print "The knn3 model thinks new_data belongs to class {}".format(pred3[0])
print "The knn5 model thinks new_data belongs to class {}".format(pred5[0])

### Visualize new point in relation to data

In [ ]:
plt.figure(figsize=(15,11))
plt.xlim(0,0.4)
plt.ylim(0,.35)
plt.scatter(X[:,0], X[:,1], c=y, s=250)
#Plot of new_data point
plt.scatter([0.18], [0.15], c="purple",marker="*", s= 1500)
plt.xlabel("Feature One")
plt.ylabel("Feature Two")
;

## Visualizing KNN

We can see the data but can we see the model itself?

In [ ]:
def plot_decision_boundary(model, X, y, n_neighbors):
    X_max = X.max(axis=0)
    X_min = X.min(axis=0)
    xticks = np.linspace(X_min[0], X_max[0], 100)
    yticks = np.linspace(X_min[1], X_max[1], 100)
    xx, yy = np.meshgrid(xticks, yticks)
    ZZ = model.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = ZZ >= 0.5
    Z = Z.reshape(xx.shape)
    fig, ax = plt.subplots()
    ax = plt.gca()
    ax.contourf(xx, yy, Z, cmap=plt.cm.bwr, alpha=0.2)
    ax.scatter(X[:,0], X[:,1], c=y,s=40, alpha=0.4)
    plt.title("Plot of {} neighbors".format(n_neighbors))
    plt.xlabel("Feature One")
    plt.ylabel("Feature Two")

In [ ]:
#Visualize the knn3 model
plot_decision_boundary(knn3, X, y, 3)

In [ ]:
#Visualize the knn3 model
plot_decision_boundary(knn5, X, y, 5)

In [ ]:
#Plot 7 neighbors
knn13 = KNeighborsClassifier(n_neighbors=13)
knn13.fit(X,y)
plot_decision_boundary(knn13, X, y, 13)

## Training vs testing data

    - Training data is the data used to create the model. The data that the machine learns from.
    - Testing data is the data used to evaluate the ability of the model. 
    - Any model can be used to classify data points it has seen, but the best models accurately classify foreign data.
    - The point of a machine learning model is keep it so you can repeatedly use it on new data.

In [ ]:
Image("train_test.jpg")
#Source: Nizam Muhammad Pega

### Load in Spotify data
A dataset of songs I like and dislike and their attributes from Spotify. 1 = like, 0 = dislike<br><br>



<b>Attributes:</b>

        Acousticness: A confidence measure from 0.0 to 1.0 of whether the track is acoustic. 1.0 represents high confidence the track is acoustic.
        
        Danceability: Danceability describes how suitable a track is for dancing based on a combination of musical elements including tempo, rhythm stability, beat strength, and overall regularity. A value of 0.0 is least danceable and 1.0 is most danceable.
        
        Instrumentalness: Predicts whether a track contains no vocals. "Ooh" and "aah" sounds are treated as instrumental in this context. Rap or spoken word tracks are clearly "vocal". The closer the instrumentalness value is to 1.0, the greater likelihood the track contains no vocal content. Values above 0.5 are intended to represent instrumental tracks, but confidence is higher as the value approaches 1.0.
        
        Valence: A measure from 0.0 to 1.0 describing the musical positiveness conveyed by a track. Tracks with high valence sound more positive (e.g. happy, cheerful, euphoric), while tracks with low valence sound more negative (e.g. sad, depressed, angry).
        
        Energy: Energy is a measure from 0.0 to 1.0 and represents a perceptual measure of intensity and activity. Typically, energetic tracks feel fast, loud, and noisy. For example, death metal has high energy, while a Bach prelude scores low on the scale. Perceptual features contributing to this attribute include dynamic range, perceived loudness, timbre, onset rate, and general entropy.
        
More details: https://developer.spotify.com/web-api/get-audio-features/

My article using this dataset: https://opendatascience.com/blog/a-machine-learning-deep-dive-into-my-spotify-data/

In [ ]:
#Load in and view subset of data
df = pd.read_pickle("songs_ML.pkl")
df.head(10)

In [ ]:
df.tail(4)

### Train KNN on the whole dataset

In [ ]:
X = df.drop("target",axis =1)
y = df.target

In [ ]:
#Find the null accuracy aka the benchmark score
y.value_counts(normalize=True)

In [ ]:
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(X,y)
score = float(knn.score(X,y))
print "The model accurately labelled {:.2f} percent of the data".format(score*100)

### Steps:

1. Split data into training and testing sets.
2. Fit knn model with 5 neighbors on the training data.
3. Make predictions on the testing data with the trained model.
4. Compare predicted labels of the testing data to its actual labels

In [ ]:
#Step 1:
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.33,
                                                    random_state=42)
#Step 2:
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)
#Step 3:
predictions = knn.predict(X_test)
#Step 4:
testing_score = accuracy_score(y_test, predictions)
# knn.score(X_test, y_test) is another way to get the accuracy score
print "The model accurately classified {:.2f} of the testing data".format(testing_score*100)

Noticeable drop in the score, how come?

## Plot the model scores for testing and training data for a range of neighbor values

In [ ]:
train_scores = []
test_scores = []
for i in range(3,25,2):
    model = KNeighborsClassifier(n_neighbors=i)
    model.fit(X_train, y_train)
    trs = model.score(X_train, y_train)
    tes = model.score(X_test, y_test)
    train_scores.append(trs)
    test_scores.append(tes)
    
plt.figure(figsize=(18,14))    
plt.plot(range(3,25,2), train_scores, label= "Training Scores",linewidth= 11)
plt.plot(range(3,25,2), test_scores, label= "Testing Scores",linewidth= 11)
plt.xlabel("N-Neighbors")
plt.ylabel("Accuracy Score")
plt.legend()